In [1]:
!pip install pytorch-lightning

In [5]:
!pip install wandb

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/6.8 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.8 MB 16.8 MB/s eta 0:00:01
   -------------------- ------------------- 3.6/6.8 MB 45.4 MB/s eta 0:00:01
   ---------------------------------------  6.8/6.8 MB 61.5 MB/s eta 0:00:01
   ---------------------------------------- 6.8/6.8 MB 54.2 MB/s eta 0:00:00
Using cached click-8.1.7-py3-none-any.whl (97 kB)
   ---------------------------------------- 0.0/207.3 kB ? eta -:--:--
   ---------------------------------------- 207.3/207.3 kB ? eta 0:00:00
   ---------------------------------------- 0.0/300.1 kB ? eta -:--:--
   ---------------------------------------- 300.1/300.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/62.7 kB ? eta -:--:--
   ---------------------------------------- 62.7/62.7 kB ? eta 0:00:00


In [2]:
# # import libraries
# import os
# import matplotlib.pyplot as plt
# import torch
# import wandb
# import torchvision
# from torch.nn import functional as F
# from torch.utils.data import DataLoader
# import pytorch_lightning as pl
# from pytorch_lightning import Trainer
# from multiprocessing import Process
# from pytorch_lightning.loggers import WandbLogger

# # Initialize W&B
# wandb.init(project="conv_vae_lstm_mnist", name="conv_vae_lstm_mnist_default")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [1]:
# import libraries
import os
import matplotlib.pyplot as plt
import torch
import wandb
import torchvision
from torch.nn import functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from multiprocessing import Process
from pytorch_lightning.loggers import WandbLogger



import argparse

def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--lr', default=1e-4, type=float, help='learning rate')
    parser.add_argument('--beta_1', type=float, default=0.9, help='decay rate 1')
    parser.add_argument('--beta_2', type=float, default=0.98, help='decay rate 2')
    parser.add_argument('--batch_size', default=16, type=int, help='batch size')
    parser.add_argument('--epochs', type=int, default=300, help='number of epochs to train for')
    parser.add_argument('--use_amp', default=True, type=bool, help='mixed-precision training')
    parser.add_argument('--n_gpus', type=int, default=2, help='number of GPUs')
    parser.add_argument('--n_hidden_dim', type=int, default=64, help='number of hidden dim for ConvLSTM layers')
    
    args = parser.parse_args([])  # Parse empty list to use defaults
    
    # Override with your desired values
    args.n_gpus = 1
    args.use_amp = True
    args.batch_size = 128
    
    return args

opt = parse_args()

import torch.nn as nn
import torch

import socket
import numpy as np
from torchvision import datasets, transforms

# from: https://github.com/edenton/svg/blob/master/data/moving_mnist.py

class MovingMNIST(object):
    """Data Handler that creates Bouncing MNIST dataset on the fly."""

    def __init__(self, train, data_root, num_digits=2, image_size=64, deterministic=True):
        path = data_root

        self.num_digits = num_digits
        self.image_size = image_size
        self.step_length = 0.1
        self.digit_size = 32
        self.deterministic = deterministic
        self.seed_is_set = True  # multi threaded loading
        self.channels = 1

        self.data = datasets.MNIST(
            path,
            train=train,
            download=True,
            transform=transforms.Compose(
                [transforms.Resize(self.digit_size),
                 transforms.ToTensor()]))

        self.N = len(self.data)

    def set_seed(self, seed):
        if not self.seed_is_set:
            self.seed_is_set = True
            np.random.seed(seed)

    def __len__(self):
        return self.N

    def __getitem__(self, index):
        self.set_seed(index)
        image_size = self.image_size
        digit_size = self.digit_size
        x = np.zeros((self.seq_len,
                      image_size,
                      image_size,
                      self.channels),
                     dtype=np.float32)
        for n in range(self.num_digits):
            idx = np.random.randint(self.N)
            digit, _ = self.data[idx]

            sx = np.random.randint(image_size - digit_size)
            sy = np.random.randint(image_size - digit_size)
            dx = np.random.randint(-4, 5)
            dy = np.random.randint(-4, 5)
            for t in range(self.seq_len):
                if sy < 0:
                    sy = 0
                    if self.deterministic:
                        dy = -dy
                    else:
                        dy = np.random.randint(1, 5)
                        dx = np.random.randint(-4, 5)
                elif sy >= image_size - 32:
                    sy = image_size - 32 - 1
                    if self.deterministic:
                        dy = -dy
                    else:
                        dy = np.random.randint(-4, 0)
                        dx = np.random.randint(-4, 5)

                if sx < 0:
                    sx = 0
                    if self.deterministic:
                        dx = -dx
                    else:
                        dx = np.random.randint(1, 5)
                        dy = np.random.randint(-4, 5)
                elif sx >= image_size - 32:
                    sx = image_size - 32 - 1
                    if self.deterministic:
                        dx = -dx
                    else:
                        dx = np.random.randint(-4, 0)
                        dy = np.random.randint(-4, 5)

                x[t, sy:sy + 32, sx:sx + 32, 0] += digit.numpy().squeeze()
                sy += dy
                sx += dx

        x[x > 1] = 1.
        return x



class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))
    

import torch
import torch.nn as nn



class VAEConvLSTM(nn.Module):
    def __init__(self, nf, in_chan, latent_dim):
        super(VAEConvLSTM, self).__init__()

        """ ARCHITECTURE 

        # Encoder (ConvLSTM)
        # Encoder Vector (final hidden state of encoder)
        # Decoder (ConvLSTM) - takes Encoder Vector as input
        # Decoder (3D CNN) - produces regression predictions for our model

        """
        
        self.latent_dim = latent_dim
        
        self.encoder_1_convlstm = ConvLSTMCell(input_dim=in_chan,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.encoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)
        #VAE components
        self.fc_mu = nn.Conv2d(nf, latent_dim, kernel_size=1)
        self.fc_logvar = nn.Conv2d(nf, latent_dim, kernel_size = 1)

        self.decoder_1_convlstm = ConvLSTMCell(input_dim=latent_dim,  # nf + 1
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_2_convlstm = ConvLSTMCell(input_dim=nf,
                                               hidden_dim=nf,
                                               kernel_size=(3, 3),
                                               bias=True)

        self.decoder_CNN = nn.Conv3d(in_channels=nf,
                                     out_channels=1,
                                     kernel_size=(1, 3, 3),
                                     padding=(0, 1, 1))


    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def encode(self, x, seq_len):
        b, _, _, h, w = x.size()
        h_t, c_t = self.encoder_1_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        h_t2, c_t2 = self.encoder_2_convlstm.init_hidden(batch_size=b, image_size=(h, w))
        
        for t in range(seq_len):
            h_t, c_t = self.encoder_1_convlstm(input_tensor=x[:, t, :, :, :], cur_state=[h_t, c_t])
            h_t2, c_t2 = self.encoder_2_convlstm(input_tensor=h_t, cur_state=[h_t2, c_t2])
        mu = self.fc_mu(h_t2)
        logvar = self.fc_logvar(h_t2)
        return mu, logvar
    
    def decode(self, z, seq_len):
        batch_size, _, height, width = z.size()
        h, c = self.init_hidden(batch_size, (height, width))
        outputs = []
        
        for t in range(seq_len):
            for i, layer in enumerate(self.decoder_layers):
                z, c[i] = layer(z, (z, c[i]))
            output = self.output_layer(z)
            outputs.append(output)
        
        return torch.stack(outputs, dim=1)
        
            
    def forward(self, x):
        seq_len = x.size(1)
        #encoding
        mu, logvar = self.encode(x, x.size(1))
        #reparameterization
        z = self.reparameterize(mu, logvar)
        #decoding
        recon_x = self.decode(z, x.size(1))
        return recon_x, mu, logvar

    def init_hidden(self, batch_size, image_size):
        h = []
        c = []
        for layer in self.encoder_layers:
            hi, ci = layer.init_hidden(batch_size, image_size)
            h.append(hi)
            c.append(ci)
        return h, c
    



##########################
######### MODEL ##########
##########################

class MovingMNISTVAE(pl.LightningModule):

    def __init__(self, hparams=None, model=None):
        super(MovingMNISTVAE, self).__init__()
        
        
        # defining the necessary parameters
        self.nf = opt.n_hidden_dim #number of filters
        self.in_chan = 1 #input channels (1 for grayscale images)
        self.path = os.getcwd() + '/data'
        self.model = model
        self.latent_dim = 10 #latent dimension for VAE
        
        self.vae_convlstm= VAEConvLSTM(nf=self.nf, in_chan=self.in_chan, latent_dim=self.latent_dim)


        '''CHECK IF YOU ACTUALLY NEED THIS
        self.lr = lr
        '''

        # logging config
        self.log_images = True

        # Training config
        self.criterion = torch.nn.MSELoss()
        self.batch_size = opt.batch_size
        self.n_steps_past = 10
        self.n_steps_ahead = 10  # 4

    def create_video(self, x, recon_x):
        # x is the input sequence
        # recon_x is the reconstructed sequence from the VAE

        # Ensure tensors are on CPU and detached from the computation graph
        x = x.cpu().detach()
        recon_x = recon_x.cpu().detach()

        # Assuming x and recon_x are of shape [batch_size, seq_len, channels, height, width]
        # We'll visualize the first item in the batch
        x = x[0]
        recon_x = recon_x[0]

        # Concatenate input and reconstruction along the sequence dimension
        video = torch.cat([x, recon_x], dim=0)

        # Calculate the absolute difference between input and reconstruction
        diff = torch.abs(x - recon_x)

        # Concatenate the difference to the video
        video = torch.cat([video, diff], dim=0)

        # Normalize to [0, 1] range for visualization
        video = (video - video.min()) / (video.max() - video.min())

        # Create a grid of images
        grid = torchvision.utils.make_grid(video, nrow=self.n_steps_past + self.n_steps_ahead)

        return grid
    
    def visualize_reconstructions(self, x, recon_x):
        # Ensure tensors are on CPU and detached from the computation graph
        x = x.cpu().detach()
        recon_x = recon_x.cpu().detach()

        # Take the first batch and first frame for visualization
        x = x[0, 0]
        recon_x = recon_x[0, 0]

        # Concatenate input and reconstruction horizontally
        comparison = torch.cat([x, recon_x], dim=2)

        # Create a grid
        grid = torchvision.utils.make_grid([comparison], nrow=1, normalize=True, scale_each=True)

        return grid 

    # OPERATIONS IN FUNCTIONS ARE MOVED AROUND A TAD BIT, HOPEFULLY SHOULDN'T AFFECT ANYTHING
    def forward(self, x):
        x = x.to(device='cuda')
        #permutted earlier because of the VAE architecture also, need to consider the probabilistic latent space now.
        x = x.permute(0, 1, 4, 2, 3)
        recon_x , mu, logvar = self.vae_convlstm(x, self.n_steps_ahead)
        return recon_x, mu, logvar
    
    #NEW LOSS FUNCTION INTRODUCED FOR VAE
    def loss_function(self, recon_x, x, mu, logvar):
        #Binary CROSS ENTROPY ONLY FOR THE MOVING MNIST DATASET BECAUSE OF 1  CHANNEL AND TWO PIXEL VALUES BLACK AND WHITE(EASY TO CLASSIFY)
        #NEED TO IMPLEMENT MSE(MEAN SQUARED ERROR) or MAE(MEAN AVERAGE ERROR) FOR 3 CHANNEL RGB IMAGES.
        BCE = F.binary_cross_entropy_with_logits(recon_x, x, reduction='sum')
        #NEED THESE LOSS FUNCTIONS WHEN USING 3 CHANNEL RGB IMAGES
        #MSE = F.mse_loss(recon_x, x, reduction='sum')
        #MAE = F.l1_loss(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1+ logvar - mu.pow(2) - logvar.exp())
        #the return would be changed when using MSE or MAE
        return BCE + KLD

    def training_step(self, batch, batch_idx):
        print("Batch structure:", type(batch), len(batch) if isinstance(batch, (list, tuple)) else batch.shape)
        # The entire sequence is in the batch
        x = batch[:, :self.n_steps_past, :, :, :]  # Input sequence
        y = batch[:, self.n_steps_past:, :, :, :]  # Target sequence
        x = x.to(device='cuda')
        
        recon_x, mu, logvar = self(x)
        loss = self.loss_function(recon_x, x, mu, logvar)

        # Log images
        if self.log_images and self.global_step % 250 == 0:
            video = self.create_video(x, recon_x)
            self.logger.experiment.add_image(
                f'epoch_{self.current_epoch}_step_{self.global_step}_reconstructions',
                video, 
                self.global_step
            )

        # Log metrics
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        # Visualize reconstructions every 1 step
        if self.global_step % 1 == 0:
            grid = self.visualize_reconstructions(x, recon_x)
            self.logger.experiment.add_image('reconstructions', grid, self.global_step)
            
            # Log to W&B
            wandb.log({"reconstructions": wandb.Image(grid)}, step=self.global_step)
        
        #few things changed around here since the loss function is different now and its vae 
        #y_hat = self.forward(x).squeeze()  # is squeeze neccessary?

        #loss = self.criterion(y_hat, y)

        # save learning_rate
        lr_saved = self.trainer.optimizers[0].param_groups[-1]['lr']
        lr_saved = torch.scalar_tensor(lr_saved).cuda()

        # save predicted images every 250 global_step
        if self.log_images:
            if self.global_step % 1 == 0:
                final_image = self.create_video(x, y_hat, y)

                self.logger.experiment.add_image(
                    'epoch_' + str(self.current_epoch) + '_step' + str(self.global_step) + '_generated_images',
                    final_image, 0)
                plt.close()
        
        
                
        

        tensorboard_logs = {'train_mse_loss': loss,
                            'learning_rate': lr_saved}

        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        x, _ = batch
        recon_x, mu, logvar = self(x)
        loss = self.loss_function(recon_x, x, mu, logvar)

        # Log metrics
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss


    def test_step(self, batch, batch_idx):
        # OPTIONAL
        x, y = batch
        y_hat = self.forward(x)
        return {'test_loss': self.criterion(y_hat, y)}


    def test_end(self, outputs):
        # OPTIONAL
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss': avg_loss}
        return {'avg_test_loss': avg_loss, 'log': tensorboard_logs}


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=opt.lr, betas=(opt.beta_1, opt.beta_2))
        #THE ONE BELOW IS SUGGESTED BY PERPLEXITY.
        #return torch.optim.Adam(self.parameters(), lr=1e-3)

    #@pl.data_loader
    def train_dataloader(self):
        train_data = MovingMNIST(
            train=True,
            data_root=self.path,
            #seq_len=self.n_steps_past + self.n_steps_ahead,
            image_size=64,
            deterministic=True,
            num_digits=2)

        train_loader = torch.utils.data.DataLoader(
            dataset=train_data,
            batch_size=self.batch_size,
            shuffle=True)

        return train_loader

    #@pl.data_loader
    def test_dataloader(self):
        test_data = MovingMNIST(
            train=False,
            data_root=self.path,
            seq_len=self.n_steps_past + self.n_steps_ahead,
            image_size=64,
            deterministic=True,
            num_digits=2)

        test_loader = torch.utils.data.DataLoader(
            dataset=test_data,
            batch_size=self.batch_size,
            shuffle=True)

        return test_loader



def run_trainer():
    # Initialize W&B
    wandb.init(project="conv_vae_lstm_mnist", name="conv_vae_lstm_mnist_default")
    
    latent_dim = 10
    conv_lstm_model = VAEConvLSTM(nf=opt.n_hidden_dim, in_chan=1, latent_dim=10)

    model = MovingMNISTVAE(model=conv_lstm_model)
    
    #using wandb logger
    wandb_logger = WandbLogger()
    

    trainer = Trainer(max_epochs=opt.epochs,
                      devices=opt.n_gpus,
                      accelerator="gpu",
                      #strategy='ddp',
                      enable_checkpointing=True,
                      precision=16 if opt.use_amp else 32,
                      logger = wandb_logger, 
                      callbacks=[
                          pl.callbacks.ModelCheckpoint(monitor='val_loss', mode='min', save_top_k=1),
                          pl.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min')
                    ]
                      
)

    trainer.fit(model)


# if __name__ == '__main__':
#     p1 = Process(target=run_trainer)                    # start trainer
#     p1.start()
#     p2 = Process(target=run_tensorboard(new_run=True))  # start tensorboard
#     p2.start()
#     p1.join()
#     p2.join()


run_trainer()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ryukijano (hack-the-thong). Use `wandb login --relogin` to force relogin


c:\Users\sc23gd\.conda\envs\pytorch\Lib\site-packages\lightning_fabric\connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\sc23gd\.conda\envs\pytorch\Lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
You are using a CUDA device ('NVIDIA GeForce RTX 4080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\sc23gd\.conda\envs\pytorch\Lib\site-packages\pytorch_light

Training: |          | 0/? [00:00<?, ?it/s]

Batch structure: <class 'torch.Tensor'> torch.Size([128, 20, 64, 64, 1])


TypeError: VAEConvLSTM.forward() takes 2 positional arguments but 3 were given